In [2]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import lxml
from collections import defaultdict, Counter
from time import sleep
from random import randint
import itertools

def get_team_year(team, year):

    date = []
    opponent = []
    game_2 = []
    home_game = []

    content = urllib.urlopen("http://www.baseball-reference.com/teams/%s/%04d-schedule-scores.shtml" % (team, year))
    s = content.read()

#     s = open("SFG.html")
    soup = BeautifulSoup(s, "lxml")

    table = soup.find("table", { "id" : "team_schedule" })

    for row in table.tbody.findAll("tr"):
        if row['class'] ==  " thead":
            continue
        
        for num, data in enumerate(row.findAll("td")):
            if num == 2:
                date.append(data['csk'])
                if data.text[-3:] == "(2)":
                    game_2.append(True)
                else: 
                    game_2.append(False) 

            elif num ==5:
                if data.text == "@":
                    home_game.append(False)
                else:
                    home_game.append(True)
                
            elif num == 6:
                opponent.append(data.text)
      
    return {'date':date, 'opponent':opponent, 'game_2':game_2, 'home_game':home_game, 
            'team': [team for num in xrange(len(date))]}

In [30]:
teams = ["LAA", "ARI", "ATL", "BAL", "BOS", "CHC",
         "CHW", "CIN", "CLE", "COL", "DET", "FLA", 
         "HOU", "KCR", "LAD", "MIL", "MIN", "NYM", 
         "NYY", "OAK", "PHI", "PIT", "SD", "SEA", 
         "STL", "TB", "TEX", "TOR", "WSN", "SFG"]

years = [2008,2009,2010,2011,2012,2013,2014,2015]

team_years = [(t,y) for t in teams for y in years]

for i in [("FLA",2012),("FLA",2013),("FLA",2014),("FLA",2015)]:
    team_years.remove(i)

for i in [("MIA",2012),("MIA",2013),("MIA",2014),("MIA",2015)]:
    team_years.append(i)

dates = []
opponents = []
game_2s = []
home_games = []
teams = []
    
for num, (t,y) in enumerate(team_years):
    sleep(randint(2,4))
    print str(num) + ": ", t, str(y)

    data = get_team_year(t,y)
    
    dates.extend(data['date'])
    opponents.extend(data['opponent'])
    game_2s.extend(data['game_2'])
    home_games.extend(data['home_game'])
    teams.extend(data['team'])

result = pd.DataFrame({'date':dates, 'opponent':opponents, 'game_2':game_2s, 'home_game':home_games, 'team':teams})

0:  LAA 2008
1:  LAA 2009
2:  LAA 2010
3:  LAA 2011
4:  LAA 2012
5:  LAA 2013
6:  LAA 2014
7:  LAA 2015
8:  ARI 2008
9:  ARI 2009
10:  ARI 2010
11:  ARI 2011
12:  ARI 2012
13:  ARI 2013
14:  ARI 2014
15:  ARI 2015
16:  ATL 2008
17:  ATL 2009
18:  ATL 2010
19:  ATL 2011
20:  ATL 2012
21:  ATL 2013
22:  ATL 2014
23:  ATL 2015
24:  BAL 2008
25:  BAL 2009
26:  BAL 2010
27:  BAL 2011
28:  BAL 2012
29:  BAL 2013
30:  BAL 2014
31:  BAL 2015
32:  BOS 2008
33:  BOS 2009
34:  BOS 2010
35:  BOS 2011
36:  BOS 2012
37:  BOS 2013
38:  BOS 2014
39:  BOS 2015
40:  CHC 2008
41:  CHC 2009
42:  CHC 2010
43:  CHC 2011
44:  CHC 2012
45:  CHC 2013
46:  CHC 2014
47:  CHC 2015
48:  CHW 2008
49:  CHW 2009
50:  CHW 2010
51:  CHW 2011
52:  CHW 2012
53:  CHW 2013
54:  CHW 2014
55:  CHW 2015
56:  CIN 2008
57:  CIN 2009
58:  CIN 2010
59:  CIN 2011
60:  CIN 2012
61:  CIN 2013
62:  CIN 2014
63:  CIN 2015
64:  CLE 2008
65:  CLE 2009
66:  CLE 2010
67:  CLE 2011
68:  CLE 2012
69:  CLE 2013
70:  CLE 2014
71:  CLE 2015
72

In [42]:
result = pd.DataFrame({'date':dates, 'opponent':opponents, 'game_2':game_2s, 'home_game':home_games, 'team':teams})

In [43]:
result2 = result

team_dict ={
    'ARI': 'ari','ATL': 'atl', 'BAL': 'bal', 'BOS': 'bos', 'CHC': 'chn',
    'CHW': 'cha', 'CIN': 'cin', 'CLE': 'cle', 'COL': 'col', 'DET': 'det', 
    'FLA': 'flo', 'HOU': 'hou', 'KCR': 'kca', 'LAA': 'ana', 'LAD': 'lan', 
    'MIA': 'mia', 'MIL': 'mil', 'MIN': 'min', 'NYM': 'nyn', 'NYY': 'nya', 
    'OAK': 'oak', 'PHI': 'phi', 'PIT': 'pit', 'SD': 'sdn', 'SEA': 'sea', 
    'SFG': 'sfn', 'STL': 'sln', 'TB': 'tba', 'TEX': 'tex', 'TOR': 'tor', 
    'WSN': 'was', 'TBR': 'tba', 'SDP': 'sdn'
}

result2['opponent'] = result2.apply(lambda row: team_dict[row['opponent']], axis=1)
result2['team'] = result2.apply(lambda row: team_dict[row['team']], axis=1)

result2.sort('date').to_csv("game_info.csv")

In [13]:
we_have = pd.read_csv("we_have.csv")
all_gids = pd.read_csv("all.csv")
# all_gids
# we_have

pd.DataFrame({'gids':list(set(all_gids['gids']) - set(we_have['gids']))}).sort('gids').to_csv("diff.csv")
# set(all_gids['gids']).intersection(set(we_have['gids']))